<a href="https://colab.research.google.com/github/ar851060/ESUN_AI_2021_winter/blob/main/2_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
coll = dict()
pred = [2,6,10,12,13,15,18,19,21,22,25,26,36,37,39,48]
cust = pd.read_csv("/content/drive/Shareddrives/統學期末比賽/Data/target.csv")['chid'].copy()
for i in range(6,25):
  df = pd.read_csv("/content/drive/Shareddrives/統學期末比賽/Data/date_"+str(i)+".csv")
  base = df.iloc[:,2:6].drop(['txn_cnt'],axis = 1)
  base = base[base['shop_tag'] != 'other']
  # base = base.loc[base['chid'].isin(cust)]
  base.sort_values(['chid','txn_amt'],ascending=[1,0],inplace=True)
  grouped = base.groupby(['chid']).head(3)
  for j in range(grouped.shape[0]):
    if int(grouped.iloc[j,1]) in pred:
      coll.setdefault(grouped.iloc[j,0], []).append(int(grouped.iloc[j,1]))
del df
del base

In [ ]:
cust = pd.read_csv("/content/drive/Shareddrives/統學期末比賽/Data/target.csv")['chid'].copy()

In [ ]:
base_ans = dict()
k = coll.keys()
for i in k:
  cand = coll[i]
  for j in range(3):
    set(cand)
    if not cand: break
    freq = max(set(cand),key = cand.count)
    base_ans.setdefault(i, []).append(int(freq))
    cand = [x for x in cand if x != freq]

In [ ]:
tot = []
for i in base_ans.values():
  tot.append(i)
fre = pd.Series(tot).value_counts()

In [ ]:
comp_3 = []
for i in fre.index:
  if len(i)>2:
    comp_3.append(i)
comp = pd.DataFrame(comp_3)

1799個沒有資訊的客人

In [ ]:
only_future = set(cust) - set(coll.keys())
for j in only_future:
  base_ans.setdefault(j, [])

In [ ]:
for idx, val in base_ans.items():
  if len(val)==1:
    base_ans[idx] = comp.loc[comp[0]==val[0]].head(1).values.tolist()[0]
  elif len(val) == 2:
    base_ans[idx] = comp.loc[(comp[0]==val[0]) & (comp[1] == val[1])].head(1).values.tolist()[0]
  elif len(val) == 0:
    base_ans[idx] = comp.head(1).values.tolist()[0]

In [ ]:
ans_df = pd.DataFrame.from_dict(base_ans)
tans = ans_df.T
tans.reset_index(inplace = True)
tans = tans.rename(columns = {"index":"chid",0:"top1",1:"top2",2:"top3"})

In [ ]:
tans.to_csv("/content/drive/Shareddrives/統學期末比賽/Data/base_answer.csv", index = False)